# Libraries

In [ ]:
import os
import sys
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

import shutil

%matplotlib inline

# import cv2
from sklearn.model_selection import train_test_split

from scipy import ndimage

from tqdm import tqdm_notebook #, tnrange
#from itertools import chain
from skimage.io import imread, imshow #, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from skimage import filters

from imgaug import augmenters as iaa

from tqdm import tqdm
from pathlib import Path

import cv2
from sklearn.model_selection import StratifiedKFold
import torch
from torch import nn
from torch import Tensor
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
import torch.backends.cudnn as cudnn
import torch.backends.cudnn
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision.transforms import ToTensor, ToPILImage, Normalize, Compose

import PIL

from datetime import datetime
import json
import gc

import time
#t_start = time.time()

# Global variable

In [ ]:
TRAIN_IMG_PATH = "../input/train/images/"
TEST_IMG_PATH = "../input/test/images/"
DEPTH_PATH = "../input/depths.csv"
TRAIN_MASK_PATH = "../input/train/masks/"
TRAIN_INFO_PATH = "../input/train.csv"

# basic parameters
IMG_ORI_SIZE = 101
IMG_TAR_SIZE = 128
SCALE = 1

# Model parameters
START_NEURONS = 16
DROPOUT_RATIO = 0.5

LOAD_CHECKPONT = False

INPUT_CHANNEL = 1

MODEL1_ADAM_LR = 0.01
MODEL1_EPOCHS = 100
MODEL1_BATCH_SIZE = 64
MODEL1_STEPS_PER_EPOCH_TRAIN = 200
MODEL1_LOSS = 'binary_crossentropy'

MODEL2_ADAM_LR = 0.01
MODEL2_EPOCHS = 100
MODEL2_BATCH_SIZE = 64
MODEL2_STEPS_PER_EPOCH_TRAIN = 200
MODEL2_LOSS = 'lovasz_loss'

# ReduceLROnPlateau parameters
MODEL1_REDUCE_FACTOR = 0.5
MODEL1_REDUCE_PATIENT = 5

MODEL2_REDUCE_FACTOR = 0.5
MODEL2_REDUCE_PATIENT = 5

# DICE_BCE_LOSS Parameters
BCE_WEIGHT = 1
DICE_WEIGHT = 0

# Augmentation Parmeters
AUG = True
FIT_METHOD = 'resize'
PAD_METHOD = 'edge'
KFOLD = False

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# encoding=utf8
import cv2
import numpy as np
from torch import nn
from functools import reduce

################################################################################
# related functions & loss functions
################################################################################


def upsample(img):
    if IMG_ORI_SIZE == IMG_TAR_SIZE:
        return img
    return cv2.resize(img, (IMG_TAR_SIZE, IMG_TAR_SIZE))


def downsample(img):
    if IMG_ORI_SIZE == IMG_TAR_SIZE:
        return img
    return cv2.resize(img, (IMG_ORI_SIZE, IMG_ORI_SIZE))


def add_depth_channels(image_array, depth):
    image_array[:,:,1] = depth
    image_array[:,:,2] = image_array[:,:,0] * image_array[:,:,1]
    return image_array


class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

        
def write_event(log, step: int, **data):
    data['step'] = step
    data['dt'] = datetime.now().isoformat()
    log.write(json.dumps(data, sort_keys=True, cls=MyEncoder))
    log.write('\n')
    log.flush()

    
def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if torch.cuda.is_available():
        return x.cuda()
    return x


def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if torch.cuda.is_available():
        return x.cpu().data.numpy()
    return x.data.numpy()
  
    
def iou_numpy(outputs, labels):
    SMOOTH = 1e-6
    labels = labels.squeeze(1)
    outputs = outputs.squeeze(1)
    
    intersection = (outputs & labels).sum((1, 2))
    union = (outputs | labels).sum((1, 2))
    
    iou = (intersection + SMOOTH) / (union + SMOOTH)
    
    thresholded = np.ceil(np.clip(20 * (iou - 0.5), 0, 10)) / 10
    
    return thresholded.mean()


def my_iou_metric(label, pred):
    return iou_numpy(pred > 0.5, label>0.5)


def my_iou_metric_2(label, pred):
    return iou_numpy(pred > 0, label>0.5)


def my_iou_metric_pad(label, pred):
    pad_size = (IMG_TAR_SIZE-SCALE*IMG_ORI_SIZE)//2
    return iou_numpy(pred[:,:,pad_size:-pad_size-1,pad_size+1:-pad_size]>0,label[:,:,pad_size:-pad_size-1,pad_size+1:-pad_size]>0.5)

    
def save_checkpoint(state, is_best, fold=0):
    filename = f'checkpoint_fold{fold}_resize.pth.tar'
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, f'model_best_fold{fold}_resize.pth.tar')
        

class EarlyStopping(object):
    def __init__(self, mode='min', min_delta=0, patience=10):
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta)

        if patience == 0:
            self.is_better = lambda a, b: True

    def step(self, metrics):
        if self.best is None:
            self.best = metrics
            return False

        if np.isnan(metrics):
            return True

        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
        else:
            self.num_bad_epochs += 1

        if self.num_bad_epochs >= self.patience:
            return True

        return False

    def _init_is_better(self, mode, min_delta):
        if mode not in {'min', 'max'}:
            raise ValueError('mode ' + mode + ' is unknown!')
        if mode == 'min':
            self.is_better = lambda a, best: a < best - min_delta
        if mode == 'max':
            self.is_better = lambda a, best: a > best + min_delta


In [ ]:
from __future__ import print_function, division

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
try:
    from itertools import  ifilterfalse
except ImportError: # py3k
    from itertools import  filterfalse
    

class DiceLoss(nn.Module):
    def __init__(self, smooth=0, eps=1e-7):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
        self.eps = eps

    def forward(self, output, target):
        return 1 - (2 * torch.sum(output * target) + self.smooth) / (
            torch.sum(output) + torch.sum(target) + self.smooth + self.eps)


class Dice_Bce_Loss(nn.Module):
    def __init__(self, smooth=0, eps=1e-7, dice_weight=0.2, 
                 dice_loss=None, bce_weight=0.9, bce_loss=None):
        super(Dice_Bce_Loss, self).__init__()
        self.smooth = smooth
        self.eps = eps
        self.dice_weight = dice_weight
        self.bce_weight = bce_weight
        self.bce_loss = bce_loss
        self.dice_loss = dice_loss
        
        if self.bce_loss is None:
            self.bce_loss = nn.BCELoss()
        if self.dice_loss is None:
            self.dice_loss = DiceLoss(smooth, eps)
            
    def forward(self, output, target):
        return self.dice_weight * self.dice_loss(output, target) + self.bce_weight * self.bce_loss(output, target)


class FocalLoss(nn.Module):
    def __init__(self, alpha=0.5, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss
        

"""
Lovasz-Softmax and Jaccard hinge loss in PyTorch
Maxim Berman 2018 ESAT-PSI KU Leuven (MIT License)
"""
def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    p = len(gt_sorted)
    gts = gt_sorted.sum()
    intersection = gts - gt_sorted.cumsum(0)
    union = gts + (1 - gt_sorted).cumsum(0)
    jaccard = 1. - intersection / union
    if p > 1: # cover 1-pixel case
        jaccard[1:p] = jaccard[1:p] - jaccard[0:-1]
    return jaccard


# --------------------------- BINARY LOSSES ---------------------------


def lovasz_hinge(logits, labels, per_image=True, ignore=None):
    """
    Binary Lovasz hinge loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      per_image: compute the loss per image instead of per batch
      ignore: void class id
    """
    if per_image:
        loss = mean(lovasz_hinge_flat(*flatten_binary_scores(log.unsqueeze(0), lab.unsqueeze(0), ignore))
                          for log, lab in zip(logits, labels))
    else:
        loss = lovasz_hinge_flat(*flatten_binary_scores(logits, labels, ignore))
    return loss


def lovasz_hinge_flat(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """
    if len(labels) == 0:
        # only void pixels, the gradients should be 0
        return logits.sum() * 0.
    signs = 2. * labels - 1.
    errors = (1. - logits * signs)
    errors_sorted, perm = torch.sort(errors, dim=0, descending=True)
    perm = perm.data
    gt_sorted = labels[perm]
    grad = lovasz_grad(gt_sorted)
    loss = torch.dot(F.relu(errors_sorted), grad)
    return loss


def flatten_binary_scores(scores, labels, ignore=None):
    """
    Flattens predictions in the batch (binary case)
    Remove labels equal to 'ignore'
    """
    scores = scores.view(-1)
    labels = labels.view(-1)
    if ignore is None:
        return scores, labels
    valid = (labels != ignore)
    vscores = scores[valid]
    vlabels = labels[valid]
    return vscores, vlabels

# --------------------------- HELPER FUNCTIONS ---------------------------

def mean(l, ignore_nan=False, empty=0):
    """
    nanmean compatible with generators.
    """
    l = iter(l)
    if ignore_nan:
        l = ifilterfalse(np.isnan, l)
    try:
        n = 1
        acc = next(l)
    except StopIteration:
        if empty == 'raise':
            raise ValueError('Empty mean')
        return empty
    for n, v in enumerate(l, 2):
        acc += v
    if n == 1:
        return acc
    return acc / n


class BCE_Lovaz_Loss(nn.Module):
    def __init__(self, per_image=True, ignore=None):
        super(BCE_Lovaz_Loss, self).__init__()
        self.per_image = per_image
        self.ignore = ignore
        
    def forward(self, logits, targets):
        loss_1 = lovasz_hinge(logits.squeeze(1), targets.squeeze(1), self.per_image, self.ignore)
        loss_2 = nn.BCEWithLogitsLoss()(logits, targets)
        loss = loss_1 + loss_2
        return loss
    
class Lovaz_Loss(nn.Module):
    def __init__(self, per_image=True, ignore=None):
        super(Lovaz_Loss, self).__init__()
        self.per_image = per_image
        self.ignore = ignore
        
    def forward(self, logits, targets):
        loss = lovasz_hinge(logits.squeeze(1), targets.squeeze(1), self.per_image, self.ignore)
        return loss

In [ ]:
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np

ia.seed(2018)

def _standardize(img):
    return (img - img.map(np.mean)) / img.map(np.std)

affine_seq = iaa.Sequential([
    # General
    iaa.SomeOf((1, 2),
               [iaa.Fliplr(0.5),
                iaa.Noop(),
                ]),
    iaa.Affine(rotate=(-5, 5), mode='reflect'),
    iaa.Crop(px=(0, 10)),
], random_order=True)

intensity_seq = iaa.Sequential([
    #iaa.Invert(0.3),
    iaa.Sometimes(0.3, iaa.ContrastNormalization((0.5, 1.5))),
    iaa.OneOf([
        iaa.Noop(),
        iaa.Sequential([
            iaa.OneOf([
                iaa.Add((-10, 10)),
                iaa.AddElementwise((-10, 10)),
                iaa.Multiply((0.95, 1.05)),
                iaa.MultiplyElementwise((0.95, 1.05)),
            ]),
        ]),
        iaa.OneOf([
            iaa.GaussianBlur(sigma=(0.0, 1.0)),
            iaa.AverageBlur(k=(2, 5)),
            #iaa.MedianBlur(k=(3, 5))
        ])
    ])
], random_order=False)

tta_intensity_seq = iaa.Sequential([
    iaa.Noop()
], random_order=False)

def resize_pad_seq(pad_size):
    seq = iaa.Sequential([
        affine_seq,
        iaa.Scale({'height': IMG_ORI_SIZE*SCALE, 'width': IMG_ORI_SIZE*SCALE}),
        iaa.Pad(px=(pad_size, 0, pad_size+1, 0), pad_mode='edge', keep_size=False),
        iaa.Pad(px=(0, pad_size, 0, pad_size+1), pad_mode='edge', keep_size=False)
    ], random_order=False)
    return seq

def resize_pad_seq_eval(pad_size):
    seq = iaa.Sequential([
        iaa.Scale({'height': IMG_ORI_SIZE*SCALE, 'width': IMG_ORI_SIZE*SCALE}),
        iaa.Pad(px=(pad_size, 0, pad_size+1, 0), pad_mode='edge', keep_size=False),
        iaa.Pad(px=(0, pad_size, 0, pad_size+1), pad_mode='edge', keep_size=False)
    ], random_order=False)
    return seq

def resize_seq():
    seq = iaa.Sequential([
        affine_seq,
        iaa.Scale({'height': IMG_TAR_SIZE, 'width': IMG_TAR_SIZE})
    ], random_order=False)
    return seq

def resize_seq_eval():
    seq = iaa.Sequential([
        iaa.Scale({'height': IMG_TAR_SIZE, 'width': IMG_TAR_SIZE})
    ], random_order=False)
    return seq

In [ ]:
# encoding=utf8
import numpy as np
import pandas as pd
from functools import partial
import cv2
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import torch.backends.cudnn as cudnn
import torch.backends.cudnn
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision.transforms import ToTensor, Normalize, Compose

## convert salt coverage to class
def cov_to_class_1(mask):
    border = 10
    outer = np.zeros((101-2*border, 101-2*border), np.float32)
    outer = cv2.copyMakeBorder(outer, border, border, border, border, borderType = cv2.BORDER_CONSTANT, value = 1)

    cover = (mask>0.5).sum()
    if cover < 8:
        return 0 # empty
    if cover == ((mask*outer) > 0.5).sum():
        return 1 #border
    if np.all(mask==mask[0]):
        return 2 #vertical

    percentage = cover/(101*101)
    if percentage < 0.15:
        return 3
    elif percentage < 0.25:
        return 4
    elif percentage < 0.50:
        return 5
    elif percentage < 0.75:
        return 6
    else:
        return 7
    
def cov_to_class_2(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i

## used to load data from data files
class my_DataLoader:
    def __init__(self, train=False, test=False, Kfold=False, test_size=0.2):
        self.test = test
        self.train = train
        self.Kfold = Kfold
        self.test_size = test_size
        self.num_fold = int(1/test_size)
            
        train_df, self.test_df = self._load_depth()
        
        if self.train:
            self._load_image_mask(train_df)
            train_df["coverage"] = train_df.masks.map(np.sum) / pow(IMG_ORI_SIZE, 2)
            train_df["coverage_class"] = train_df.masks.map(cov_to_class_1)
            self.x_train, self.x_valid, self.y_train, self.y_valid = self._get_train_test_split(train_df, self.Kfold, self.num_fold)
            
        if self.test:
            test_df['images'] = self._load_image_test(self.test_df)
            self.x_test = np.array(self.test_df.images.tolist()).reshape(-1, IMG_ORI_SIZE, IMG_ORI_SIZE, 1)

    @staticmethod
    def _load_image_mask(train_df):
        # load image data & mask data
        train_df['images'] = [np.array(cv2.imread(TRAIN_IMG_PATH + "{}.png".format(idx), 0)) for idx in tqdm_notebook(train_df.index)]
        train_df['masks'] = [np.array(cv2.imread(TRAIN_MASK_PATH + "{}.png".format(idx), 0)) for idx in tqdm_notebook(train_df.index)]
        # Normalize image vectors
        #train_df['images'] /= 255
        #train_df['masks'] /= 255
        
    @staticmethod
    def _load_image_test(test_df):
        return [np.array(cv2.imread(TEST_IMG_PATH + "{}.png".format(idx), 0)) for idx in tqdm_notebook(test_df.index)]

    @staticmethod
    def _load_depth():
        train_df = pd.read_csv(TRAIN_INFO_PATH, index_col="id", usecols=[0])
        depths_df = pd.read_csv(DEPTH_PATH, index_col="id")
        depths_df['z'] = depths_df['z'].astype('float')
        train_df = train_df.join(depths_df)
        test_df = depths_df[~depths_df.index.isin(train_df.index)]
        return train_df, test_df

    ## get train & validation split stratified by salt coverage
    @staticmethod
    def _get_train_test_split(train_df, Kfold, num_fold):
        x_train, x_valid, y_train, y_valid = [], [], [], []
        skf = StratifiedKFold(n_splits=num_fold, random_state=1234, shuffle=True)
        for train_index, valid_index in skf.split(train_df.index.values, train_df.coverage_class):
            x_tr = np.array(train_df.images[train_index].tolist()).reshape(-1, IMG_ORI_SIZE, IMG_ORI_SIZE, 1)
            x_tr = np.append(x_tr, [np.fliplr(x) for x in x_tr], axis=0)
            x_train.append(x_tr)
            x_valid.append(np.array(train_df.images[valid_index].tolist()).reshape(-1, IMG_ORI_SIZE, IMG_ORI_SIZE, 1))
            y_tr = np.array(train_df.masks[train_index].tolist()).reshape(-1, IMG_ORI_SIZE, IMG_ORI_SIZE, 1)
            y_tr = np.append(y_tr, [np.fliplr(y) for y in y_tr], axis=0)
            y_train.append(y_tr)
            y_valid.append(np.array(train_df.masks[valid_index].tolist()).reshape(-1, IMG_ORI_SIZE, IMG_ORI_SIZE, 1))
            if not Kfold:
                break
        return x_train, x_valid, y_train, y_valid

    def get_train(self):
        return self.x_train, self.y_train
    
    def get_valid(self):
        return self.x_valid, self.y_valid

    def get_test_x(self):
        return self.x_test

    def get_test_df(self):
        return self.test_df
    
class ShipDataset(Dataset):
    def __init__(self, data, transform=None, mode='train'):
        if mode == 'train' or mode == 'valid':
            self.x = data[0]
            self.y = data[1]
        elif mode == 'test':
            self.data = data
        else:
            raise RuntimeError('MODE_ERROR')
        self.transform = transform
        self.mode = mode
        self.pad_method = PAD_METHOD
        self.pad_size = (IMG_TAR_SIZE-IMG_ORI_SIZE)//2
        
        if FIT_METHOD == 'resize_pad':
            self.aug_func_eval = partial(resize_pad_seq_eval, self.pad_size)
        elif FIT_METHOD == 'resize':
            self.aug_func_eval = resize_seq_eval
            
        if AUG:
            if FIT_METHOD == 'resize_pad':
                self.aug_func = partial(resize_pad_seq, self.pad_size)
            elif FIT_METHOD == 'resize':
                self.aug_func = resize_seq
        
        if INPUT_CHANNEL == 3:
            self.depth = np.tile(np.linspace(0,1,IMG_TAR_SIZE),[IMG_TAR_SIZE,1]).T

    def __len__(self):
        if self.mode == 'train' or self.mode == 'valid':
            return len(self.x)
        elif self.mode == 'test':
            return len(self.data)
        else:
            raise RuntimeError('MODE_ERROR')
               
    def __getitem__(self, idx):
        if self.mode == 'train':
            if AUG:
                resize_seq_det = self.aug_func().to_deterministic()
                new_x_batch = resize_seq_det.augment_image(self.x[idx])
                new_x_batch = intensity_seq.augment_image(new_x_batch)/255
                new_y_batch = resize_seq_det.augment_image(self.y[idx])/255
            else:
                resize_seq_det = self.aug_func_eval().to_deterministic()
                new_x_batch = resize_seq_det.augment_image(self.x[idx])/255
                new_y_batch = resize_seq_det.augment_image(self.y[idx])/255
            if INPUT_CHANNEL == 3:
                new_x_batch = np.tile(new_x_batch,(1,1,3))
                new_x_batch = add_depth_channels(new_x_batch, self.depth)
            return new_x_batch, new_y_batch
        elif self.mode == 'valid':
            resize_seq_det = self.aug_func_eval().to_deterministic()
            new_x_batch = resize_seq_det.augment_image(self.x[idx])/255
            new_y_batch = resize_seq_det.augment_image(self.y[idx])/255
            if INPUT_CHANNEL == 3:
                new_x_batch = np.tile(new_x_batch,(1,1,3))
                new_x_batch = add_depth_channels(new_x_batch, self.depth)
            return new_x_batch, new_y_batch
        elif self.mode == 'test':
            resize_seq_det = self.aug_func_eval()
            test_data = resize_seq_det.augment_image(self.data[idx])/255
            if INPUT_CHANNEL == 3:
                test_data = np.tile(test_data,(1,1,3))
                new_x_batch = add_depth_channels(test_data, self.depth)
            return test_data
        else:
            raise RuntimeError('MODE_ERROR')
            
def make_loader(data, batch_size, num_workers=4, shuffle=False, transform=None, mode='train'):
        return DataLoader(
            dataset=ShipDataset(data, transform=transform, mode=mode),
            shuffle=shuffle,
            num_workers = num_workers,
            batch_size = batch_size,
            pin_memory=torch.cuda.is_available()
        )

In [ ]:
dl = my_DataLoader(train=True, Kfold=KFOLD)
x_train, y_train = dl.get_train()
x_valid, y_valid = dl.get_valid()

#Data augmentation
#x_train = np.append(x_train, [np.fliplr(x) for x in x_train], axis=0)
#y_train = np.append(y_train, [np.fliplr(x) for x in y_train], axis=0)

In [ ]:
from torch import nn
from torch.nn import functional as F
import torch
from torchvision import models
import torchvision.models.resnet
from torchvision.models.resnet import BasicBlock, Bottleneck
import torch.utils.model_zoo as model_zoo

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}
"""
This script has been taken (and modified) from :
https://github.com/ternaus/TernausNet
@ARTICLE{arXiv:1801.05746,
         author = {V. Iglovikov and A. Shvets},
          title = {TernausNet: U-Net with VGG11 Encoder Pre-Trained on ImageNet for Image Segmentation},
        journal = {ArXiv e-prints},
         eprint = {1801.05746}, 
           year = 2018
        }
"""
class ConvBn2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding):
        super().__init__()
        self.conv = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding),
                                  nn.BatchNorm2d(out_channels),
                                  )

    def forward(self, x):
        return self.conv(x)


class NoOperation(nn.Module):
    def forward(self, x):
        return x
    
    
class CSE(nn.Module):
    def __init__(self, in_ch, r=2):
        super(CSE, self).__init__()

        self.linear_1 = nn.Linear(in_ch, in_ch//r)
        self.linear_2 = nn.Linear(in_ch//r, in_ch)

        
    def forward(self, x):
        input_x = x

        x = x.view(*(x.shape[:-2]),-1).mean(-1)
        x = F.relu(self.linear_1(x), inplace=True)
        x = self.linear_2(x)
        x = x.unsqueeze(-1).unsqueeze(-1)
        x = torch.sigmoid(x)

        x = input_x * x

        return x


class SSE(nn.Module):
    def __init__(self, in_ch):
        super(SSE, self).__init__()

        self.conv = nn.Conv2d(in_ch, in_ch, kernel_size=1, stride=1)

    def forward(self, x):
        input_x = x

        x = self.conv(x)
        x = torch.sigmoid(x)

        x = input_x * x

        return x


class SCSE(nn.Module):
    def __init__(self, in_ch, r=2):
        super(SCSE, self).__init__()

        self.cSE = CSE(in_ch, r)
        self.sSE = SSE(in_ch)

    def forward(self, x):
        cSE = self.cSE(x)
        sSE = self.sSE(x)

        x = cSE + sSE

        return x
    
    
class Decoder(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels):
        super(Decoder, self).__init__()
        self.conv1 = ConvBn2d(in_channels, middle_channels, kernel_size=3, padding=1)
        self.conv2 = ConvBn2d(middle_channels, out_channels, kernel_size=3, padding=1)
        self.SCSE = SCSE(out_channels)
        
    def forward(self, x, e=None):
        x = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False)
        if e is not None:
            x = torch.cat([x,e], 1)
        x = F.relu(self.conv1(x), inplace=True)
        x = F.relu(self.conv2(x), inplace=True)
        x = self.SCSE(x)

        return x


class UNetResNet(nn.Module):

    def __init__(self, encoder_depth, num_classes, dropout_2d=0.2, pretrained=True):
        super().__init__()
        self.num_classes = num_classes
        self.dropout_2d = dropout_2d

        if encoder_depth == 34:
            self.resnet = torchvision.models.resnet34(pretrained=pretrained)
            bottom_channel_nr = 512
        elif encoder_depth == 101:
            self.resnet = torchvision.models.resnet101(pretrained=pretrained)
            bottom_channel_nr = 2048
        elif encoder_depth == 152:
            self.resnet = torchvision.models.resnet152(pretrained=pretrained)
            bottom_channel_nr = 2048
        else:
            raise NotImplementedError('only 34, 101, 152 version of Resnet are implemented')

        self.encoder1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=1, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.encoder2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            self.resnet.layer1,
        )
        self.encoder3 = self.resnet.layer2
        self.encoder4 = self.resnet.layer3
        self.encoder5 = self.resnet.layer4
        
        self.center = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            )

        self.decoder5 = Decoder(256+512, 512, 64)
        self.decoder4 = Decoder( 64+256, 256, 64)
        self.decoder3 = Decoder( 64+128, 128, 64)
        self.decoder2 = Decoder( 64+ 64,  64, 64)
        self.decoder1 = Decoder( 64+ 64,  32, 64)
        
        self.logit_pixel  = nn.Sequential(
            nn.Conv2d(320, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d( 64,  1, kernel_size=1, padding=0),
        )

        self.logit_image = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 1),
        )

    def forward(self, x):
        mean= [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        x = torch.cat([
            (x-mean[2])/std[2],
            (x-mean[1])/std[1],
            (x-mean[0])/std[0],
        ],1)
        """
        if INPUT_CHANNEL == 1:
            x = torch.cat([x,x,x],1)
        """
        
        e1 = self.encoder1(x) #;print('e1', e1.size())
        e2 = self.encoder2(e1)#;print('e2', e2.size())
        e3 = self.encoder3(e2)#;print('e3', e3.size())
        e4 = self.encoder4(e3)#;print('e4', e4.size())
        e5 = self.encoder5(e4)#;print('e5', e5.size())
        
        f = self.center(e5)        #;print('f', f.size())
        
        d5 = self.decoder5(f,e5)   #;print('d5', d5.size())
        d4 = self.decoder4(d5,e4)  #;print('d4', d4.size())
        d3 = self.decoder3(d4,e3)  #;print('d3', d3.size())
        d2 = self.decoder2(d3,e2)  #;print('d2', d2.size())
        d1 = self.decoder1(d2,e1)  #;print('d1', d1.size())
        
        #hyper column
        f = torch.cat((
            d1,
            F.interpolate(d2, scale_factor= 2, mode='bilinear', align_corners=False),
            F.interpolate(d3, scale_factor= 4, mode='bilinear', align_corners=False),
            F.interpolate(d4, scale_factor= 8, mode='bilinear', align_corners=False),
            F.interpolate(d5, scale_factor=16, mode='bilinear', align_corners=False),
        ), 1)
        
        f = F.dropout2d(f, p=self.dropout_2d)
        logit = self.logit_pixel(f)
        return logit

In [ ]:
def validation(model: nn.Module, criterion, valid_loader):
    model.eval()
    losses = []
    iou = []
    with torch.no_grad():
        for inputs, targets in valid_loader:
            inputs = inputs.permute(0,3,1,2).type(torch.FloatTensor).to(device)
            targets = targets.permute(0,3,1,2).type(torch.FloatTensor).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            losses += [loss.item()]
            iou += [my_iou_metric_2(get_numpy(targets), get_numpy(outputs))]

        valid_loss = np.mean(losses)  # type: float

        valid_iou = np.mean(iou)

        metrics = {'val_loss': valid_loss, 'val_iou': valid_iou}
    return metrics

def train(model, criterion, train_loader, optimizer, epoch, scheduler,
          report_each=10, valid_iou=0, fold=None):
        model.train()
        random.seed()
        scheduler.step(valid_iou)
        losses = []
        ious = []
        tl = train_loader
        
        try:
            mean_loss = 0
            mean_iou = 0
            for i, (inputs, targets) in enumerate(tl):
                inputs = inputs.permute(0,3,1,2).type(torch.FloatTensor).to(device)
                targets = targets.permute(0,3,1,2).type(torch.FloatTensor).to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                optimizer.zero_grad()
                batch_size = inputs.size(0)
                loss.backward()
                optimizer.step()
                losses += [loss.item()]
                ious += [my_iou_metric_2(get_numpy(targets), get_numpy(outputs))]
                mean_loss = np.mean(losses[-report_each:])
                mean_iou = np.mean(ious[-report_each:])
                
                for param_group in optimizer.param_groups:
                    lr = param_group['lr']

                if i % report_each == 0:
                    print('Epoch: [{0}][{1}/{2}]\t'
                          'LR: {learning_rate:.5f}\t'
                          'Loss {loss:.4f} ({loss_avg:.4f})\t'
                          'IOU {iou:.3f} ({iou_avg:.3f})'.format(
                           epoch, i, len(tl), learning_rate=lr, loss=losses[-1], loss_avg=mean_loss, iou=ious[-1], iou_avg=mean_iou))

            metrics = {'train_loss': mean_loss, 'train_iou': mean_iou}
            return metrics
        
        except KeyboardInterrupt:
            print('Ctrl+C, saving snapshot')
            print('done.')
            return

In [ ]:
import torch.optim as optim
from torch.autograd import Variable

criterion = FocalLoss(alpha = 0.5, logits = True)
criterion = criterion.to(device) 
early_stop = EarlyStopping(mode='max', min_delta=0, patience=10)

In [ ]:
fold = 0
for x_train_f, y_train_f, x_valid_f, y_valid_f in zip(x_train, y_train, x_valid, y_valid):
    train_loader = make_loader((x_train_f, y_train_f), num_workers=0, batch_size=32, shuffle=True)
    valid_loader = make_loader((x_valid_f, y_valid_f), num_workers=0, batch_size=64, mode='valid')

    res_unet = UNetResNet(34, 1, dropout_2d=0.5, pretrained=True)
    res_unet = res_unet.to(device)

    early_stop = EarlyStopping(mode='max', min_delta=0, patience=10)
    #model = nn.DataParallel(model, device_ids=None)

    n_epochs = 45
    start_epoch = 0
    report_each = 20
    valid_losses = []
    valid_ious = []
    train_losses = []
    train_ious = []
    valid_iou = 0
    valid_loss = 0
    best_iou = 0
            
    optimizer= Adam(res_unet.parameters(), lr=0.002)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', verbose=True, patience=5, factor=0.5, min_lr=0.0001)
        
    print('Fold: [{0}]\t'.format(fold))
    for epoch in range(start_epoch, start_epoch + n_epochs + 1):
        time0 = time.time()
        train_metrics = train(res_unet, criterion, train_loader, optimizer, 
                              epoch, scheduler, report_each, valid_iou)
        valid_metrics = validation(res_unet, criterion, valid_loader)

        train_loss = train_metrics['train_loss']
        train_iou = train_metrics['train_iou']
        train_losses += [train_loss]
        train_ious += [train_iou]
        valid_loss = valid_metrics['val_loss']
        valid_iou = valid_metrics['val_iou']
        valid_losses += [valid_loss]
        valid_ious += [valid_iou]
        is_best = best_iou < valid_iou
        best_iou = max(valid_iou, best_iou)
        print('Epoch: [{0}][Validation]\t' 
              'Val_Loss: {val_loss:.5f}\t' 
              'Val_IOU: {val_iou:.5f}\t'
              'Best Val_IOU: {best_iou:.5f}'.format(epoch, val_loss=valid_loss, val_iou=valid_iou, best_iou=best_iou))
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': res_unet.state_dict(),
            'best_iou': best_iou,
            'optimizer': optimizer.state_dict(),
        }, is_best, fold=fold)
        time1 = time.time()
        print('epoch time: ', time1-time0)
    fold += 1
    if early_stop.step(valid_iou):
        break